In [ ]:
#!pip install mysql-connector-python

import os
import pandas as pd
import mysql.connector
from  mysql.connector import errorcode
import ast

In [ ]:
def limpieza_puntuaciones():
    """
    Esta función transforma un csv en una lista de tuplas de puntuaciones por pelicula.
    
    Args:
    - No recibe parametros.

    Returns:
    - devuelve una lista de tuplas (id_pelicula, nombre_pelicula, puntos_imdb, puntos_rotten).
    """

    lista_tuplas_puntuaciones = []

    for i in range(1990,2024):
        
        try:
            df_puntos = pd.read_csv(f'datos/puntuaciones/puntuaciones_{i}_1000.csv')
            #df_puntuaciones = df_puntos.dropna(how='any', axis=0) 


            for fila in df_puntos.values:
                lista_tuplas_puntuaciones.append(tuple(fila))
        except:
            print(f"puntuaciones_{i}_1000.csv - no existe")            

    lista_final_puntuaciones = list(set(lista_tuplas_puntuaciones)) 

    return lista_final_puntuaciones


In [ ]:
def insertar_puntuaciones(lista):
    """
    Esta función toma una lista de tuplas y la inserta en una tabla de la bbdd
    
    Args:
    - lista: listade tuplas donde cada tupla incluye los datos de las puntuaciones de una película (d_pelicula, nombre_pelicula, puntos_imdb, puntos_rotten)

    Returns:
    - devuelve el número de registros insertados

    """
    contrasena = os.environ.get('contraseña')

    #Tabla puntuaciones
    cnx = mysql.connector.connect(user="root",
                                password=contrasena,
                                host="localhost", database = "bho_peliculas_3")

    mycursor = cnx.cursor()

    sql_puntuaciones = "INSERT INTO puntuaciones (id_pelicula, nombre_pelicula, puntos_imdb, puntos_rotten) VALUES (%s, %s, %s, %s)"

    try:
        mycursor.executemany(sql_puntuaciones, lista)
        cnx.commit()
        print(mycursor.rowcount, "registros insertados")
    except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)
                print("peliculas - error inserción")

    cnx.close()    

    return  mycursor.rowcount             

In [ ]:
#llamar a las funciones para limpiar los datos e insertarlos en la bbdd
insertar_puntuaciones(limpieza_puntuaciones())